# Cervical Cancer Screening UDS Prototype (2025)

This notebook ingests:
- Scanned PAP smear reports (PDF) via OCR
- HPV test results (PDF) via OCR
- Cervical screening CSV extract

It outputs per-patient cervical cancer screening numerator status for 2025 with an auditor-friendly evidence string.

## UDS 2025 Cervical Cancer Screening Rules:
- **PAP smear alone**: Within 3 years (2023-2025)
- **HPV test alone**: Within 5 years (2021-2025)
- **Co-testing (PAP + HPV)**: Within 5 years (2021-2025)
- Age range: Typically 21-64 years (verify with your UDS specifications)

In [50]:
import os, re, json, datetime as dt
import pandas as pd
from dateutil.relativedelta import relativedelta
import pytesseract
from pdf2image import convert_from_path

In [51]:
import glob, os

# Auto-discover cervical cancer screening PDFs
PDF_GLOBS = [
    "pap_pdf/*pap*.pdf",
    "pap_pdf/*cervical*.pdf",
    "pap_pdf/*hpv*.pdf",
    "pap_pdf/*papanicolaou*.pdf",
    "pap_pdf/*cytology*.pdf",
]
PDF_PATHS = sorted({p for g in PDF_GLOBS for p in glob.glob(g)})

CERVICAL_CSV_PATH = "csv_data/cervical_screening.csv"
REPORTING_YEAR = 2025

print(f"Found {len(PDF_PATHS)} PDFs:")
for p in PDF_PATHS:
    print(" -", os.path.basename(p))
print("Cervical CSV:", os.path.basename(CERVICAL_CSV_PATH), "exists:", os.path.exists(CERVICAL_CSV_PATH))

Found 16 PDFs:
 - 03_03_1970_pap.pdf
 - 117433_pap.pdf
 - 118815_pap.pdf
 - 135613_pap_note.pdf
 - 144061_pap.pdf
 - 151159_pap.pdf
 - 158353_pap.pdf
 - 159266_pap_hpv.pdf
 - 176310_pap.pdf
 - 176631_pap_note.pdf
 - 186419_pap.pdf
 - 36387_pap.pdf
 - 38837_pap.pdf
 - 58437_pap.pdf
 - 58437_pap_referral.pdf
 - 97024_pap_note.pdf
Cervical CSV: cervical_screening.csv exists: False


In [ ]:
import os, re, datetime as dt
from dataclasses import dataclass
from typing import Optional, Literal, Dict, Any, List, Tuple

DATE_PAT = re.compile(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})')
# Match dates with month names like "April 02, 2018" or "April 2, 2018"
MONTH_NAME_PAT = re.compile(r'\b(January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)\.?\s+(\d{1,2}),?\s+(\d{4})\b', re.IGNORECASE)

def _to_iso(m:int,d:int,y:int) -> Optional[str]:
    if y < 100:
        y = 2000 + y if y < 50 else 1900 + y
    try:
        return dt.date(y,m,d).isoformat()
    except Exception:
        return None

def parse_dates(text: str) -> List[Tuple[str, str]]:
    out=[]
    # Extract numeric dates like 4/2/2018
    for m,d,y in DATE_PAT.findall(text or ""):
        iso = _to_iso(int(m), int(d), int(y))
        if iso:
            out.append((iso, f"{m}/{d}/{y}"))
    
    # Extract dates with month names like "April 02, 2018"
    month_map = {
        'jan': 1, 'january': 1, 'feb': 2, 'february': 2, 'mar': 3, 'march': 3,
        'apr': 4, 'april': 4, 'may': 5, 'jun': 6, 'june': 6,
        'jul': 7, 'july': 7, 'aug': 8, 'august': 8, 'sep': 9, 'sept': 9, 'september': 9,
        'oct': 10, 'october': 10, 'nov': 11, 'november': 11, 'dec': 12, 'december': 12
    }
    for month_name, day, year in MONTH_NAME_PAT.findall(text or ""):
        month_num = month_map.get(month_name.lower())
        if month_num:
            iso = _to_iso(month_num, int(day), int(year))
            if iso:
                out.append((iso, f"{month_name} {day}, {year}"))
    
    return out

def looks_like_nonclinical_line(line: str) -> bool:
    l=(line or "").lower()
    if "electronically signed" in l or "signed by" in l:
        return True
    if "dob" in l or "date of birth" in l:
        return True
    if "data:text" in l or "base64" in l:
        return True
    # Filter out fax cover sheet lines
    if "from: fax" in l or ("from:" in l and "to:" in l and "page:" in l):
        return True
    return False

def filename_patient_id(path: str) -> str:
    base=os.path.basename(path)
    m=re.match(r'^(\d+)', base)
    return m.group(1) if m else base

import pdfplumber
from pdf2image import convert_from_path
import pytesseract

KEYWORDS = ["pap","papanicolaou","cervical","cytology","hpv","human papillomavirus","squamous","ascus","lsil","hsil"]

# POPPLER PATH: Uncomment and set this if poppler is not in your system PATH
POPPLER_PATH = r"C:\poppler\Library\bin"

# TESSERACT PATH: Set path to tesseract executable
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\jloya\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
#POPPLER_PATH = None  # Set to None to use system PATH

def extract_first_page_text(pdf_path: str, ocr_dpi: int = 300, min_chars: int = 80, max_pages: int = 5) -> str:
    """Extract text from PDF, checking multiple pages if needed. Uses OCR fallback for scanned documents."""
    all_text = ""
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            pages_to_check = min(max_pages, total_pages)
            
            # Try extracting from first few pages
            for page_num in range(pages_to_check):
                page_text = (pdf.pages[page_num].extract_text() or "")
                all_text += page_text + "\n"
                
                # Check if we have enough clinical content
                text_l = all_text.lower()
                keyword_hit = any(k in text_l for k in KEYWORDS)
                if len(all_text.strip()) >= min_chars and keyword_hit:
                    # Found good content, stop here
                    return all_text
    except Exception:
        pass

    # If we didn't get good text, try OCR on first few pages
    if len(all_text.strip()) < min_chars or not any(k in all_text.lower() for k in KEYWORDS):
        try:
            ocr_combined = ""
            # OCR up to max_pages or until we find clinical content
            for page_num in range(1, min(max_pages + 1, 15)):  # Cap at 15 pages max
                pages = convert_from_path(
                    pdf_path, 
                    dpi=ocr_dpi, 
                    first_page=page_num, 
                    last_page=page_num,
                    poppler_path=POPPLER_PATH
                )
                if pages:
                    ocr_text = pytesseract.image_to_string(pages[0], timeout=12)
                    ocr_combined += ocr_text + "\n"
                    
                    # Check if we found clinical content
                    ocr_l = ocr_combined.lower()
                    keyword_hit = any(k in ocr_l for k in KEYWORDS)
                    if len(ocr_combined.strip()) >= min_chars and keyword_hit:
                        # Found good OCR content, stop here
                        return ocr_combined
                        
            # Return whatever OCR we got if it's better than pdfplumber
            if len(ocr_combined.strip()) > len(all_text.strip()):
                return ocr_combined
        except Exception as e:
            pass
    
    # Return whatever we got, even if minimal
    return all_text or ""

    # OCR fallback - CRITICAL for scanned PDFs and lab reports
    # Use higher DPI (300) for poor quality scans
    try:
        pages = convert_from_path(
            pdf_path, 
            dpi=ocr_dpi, 
            first_page=1, 
            last_page=1,
            poppler_path=POPPLER_PATH
        )
        if pages:
            ocr_text = pytesseract.image_to_string(pages[0], timeout=12)
            if ocr_text and len(ocr_text.strip()) > len(text.strip()):
                return ocr_text
    except Exception as e:
        # OCR failed - likely poppler/tesseract not installed
        # For header-only PDFs, warn user
        if header_only:
            print(f"⚠️ {pdf_path}: Appears to be a lab report requiring OCR. Install Tesseract for accurate extraction.")
        pass
    
    # Return whatever we got, even if minimal
    return text or ""

EventType = Literal["PAP_SMEAR","HPV_TEST","CO_TEST"]

@dataclass
class ScreeningEvent:
    patient_id: str
    event_type: EventType
    event_date: Optional[str]
    source: Literal["pdf","structured_csv"]
    confidence: float
    needs_review: bool
    evidence: Dict[str, Any]

def classify_doc(text: str) -> Optional[EventType]:
    """Classify cervical cancer screening document type."""
    t=(text or "").lower()
    
    # Check for both PAP and HPV mentions (co-testing)
    has_pap = bool(re.search(r'\b(pap|papanicolaou|cytology|squamous)\b', t))
    has_hpv = bool(re.search(r'\b(hpv|human papillomavirus)\b', t))
    
    if has_pap and has_hpv:
        return "CO_TEST"
    elif has_hpv:
        return "HPV_TEST"
    elif has_pap:
        return "PAP_SMEAR"
    
    # Check for cervical-related terms
    if "cervical" in t and ("screen" in t or "specimen" in t or "collection" in t):
        return "PAP_SMEAR"  # Default to PAP if cervical screening mentioned
    
    return None

def extract_labeled_dates(text: str) -> Dict[str, List[str]]:
    labels = {"specimen":[],"collection":[],"received":[],"result":[],"procedure":[],"other":[]}
    for line in (text or "").splitlines():
        if not line.strip():
            continue
        if looks_like_nonclinical_line(line):
            continue
        ll=line.lower()
        label="other"
        
        # PRIORITY 1: Collection/Specimen dates (most specific patterns first)
        # Match "Collection Date" or font-corrupted variants with NUL bytes: "Collec\x00on Date"
        # PDF font encoding corrupts "Collection" by inserting NUL (0x00) bytes
        if re.search(r'\bcollec[\s\x00]*[ti]?on\b.*\bdate\b', ll) or re.search(r'\bcollect\b.*\bdate\b', ll):
            label="collection"
        # Match "Order Date" - typically the date specimen was ordered/collected
        elif re.search(r'\border\b.*\bdate\b', ll):
            label="collection"
        # Match "Final, MM/DD/YYYY" pattern - the date after Final is often the collection date
        elif re.search(r'\bfinal\s*,\s*\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', ll):
            label="collection"
        # Match "Specimen Date" or "Date of Specimen" or "specimen coll date"
        elif re.search(r'\bspecimen\b.*\b(coll|collection)\b.*\bdate\b', ll):
            label="collection"
        elif re.search(r'\b(specimen|sample)\b.*\bdate\b|date.*\b(specimen|sample)\b', ll):
            label="specimen"
        # Pattern for "collected on/at MM/DD/YYYY" or "obtained: MM/DD/YYYY"
        elif re.search(r'\b(collected|obtained)\b.*(on|at|:)', ll):
            label="collection"
        # LAB REPORT PATTERN: "Approved" followed by date (common in LabCorp/Quest reports)
        # These are actually collection dates, not result dates
        elif re.search(r'\bapproved\b', ll):
            label="collection"
            
        # PRIORITY 2: Other clinical dates
        elif re.search(r'procedure\s*date|date\s*of\s*procedure|date\s*of\s*service|\bdos\b', ll):
            label="procedure"
        # Match "specimen recv date" or "specimen received date"
        elif re.search(r'\bspecimen\b.*\b(recv|received)\b.*\bdate\b', ll):
            label="received"
        elif re.search(r'\breceived\b.*\bdate\b|date.*\breceived\b', ll):
            label="received"
        # Match "specimen reported date"
        elif re.search(r'\bspecimen\b.*\breported\b.*\bdate\b', ll):
            label="result"
        # Match "Report:" or "Reported:" patterns
        elif re.search(r'\breport(ed)?\s*:', ll):
            label="result"
        elif re.search(r'\bresult\b.*\bdate\b|date.*\bresult\b|\breported\b.*\bdate\b|date.*\breported\b', ll):
            label="result"
        # Generic "result" or "final" without date context - less specific
        elif re.search(r'\bresult\b|\breported\b|\bfinal\b', ll):
            label="result"
            
        for iso,_ in parse_dates(line):
            labels[label].append(iso)
    for k,v in labels.items():
        seen=set(); nv=[]
        for d in v:
            if d not in seen:
                nv.append(d); seen.add(d)
        labels[k]=nv
    return labels

def is_reasonable_procedure_date(date_iso: str) -> bool:
    """Filter out dates that are clearly not procedure dates (e.g., DOBs from 1950s, future dates)"""
    try:
        year = int(date_iso[:4])
        current_year = dt.date.today().year
        # Cervical screening should be in a reasonable range
        # Allow 1995 to current year (no future dates - likely OCR errors)
        return 1995 <= year <= current_year
    except:
        return False

def choose_best_date(event_type: EventType, labeled: Dict[str,List[str]], all_dates: List[str]) -> Tuple[Optional[str], float, bool, str]:
    # For all cervical screening types, prefer collection/specimen date
    pref="collection"
    
    # Filter out invalid dates (DOBs, future dates) from all categories first
    for key in labeled:
        labeled[key] = [d for d in labeled[key] if is_reasonable_procedure_date(d)]
    
    candidates = labeled.get("collection", []).copy()
    if not candidates:
        candidates = labeled.get("specimen",[])
    if not candidates:
        # For fallback, prefer result (report date) over received date for accuracy
        candidates = labeled.get("result",[]) + labeled.get("procedure",[]) + labeled.get("received",[]) + labeled.get("other",[])
    if not candidates and all_dates:
        candidates = [d for d in all_dates if is_reasonable_procedure_date(d)]
        candidates = list(dict.fromkeys(candidates))

    if not candidates:
        return None, 0.2, True, "no date found"

    best = max(candidates)
    conf = 0.9 if ((labeled.get("collection") and best in labeled["collection"]) or 
                   (labeled.get("specimen") and best in labeled["specimen"])) else 0.65
    needs_review = False
    rationale = f"picked {best} from {pref if conf>0.8 else 'fallback'}"

    distinct = sorted(set(sum(labeled.values(), [])))
    if conf < 0.8 and len(distinct) >= 2:
        needs_review = True
        conf = min(conf, 0.55)
        rationale += "; multiple conflicting dates"

    return best, conf, needs_review, rationale

def extract_event_from_pdf(pdf_path: str, text: str) -> ScreeningEvent:
    pid = filename_patient_id(pdf_path)
    filename = os.path.basename(pdf_path).lower()
    
    # Try to classify from text content first
    et = classify_doc(text)
    if et is None:
        # Fallback to filename-based classification
        if "hpv" in filename:
            et = "HPV_TEST"
        elif "pap" in filename or "cervical" in filename or "cytology" in filename:
            et = "PAP_SMEAR"
        else:
            et = "PAP_SMEAR"  # Default assumption
    
    labeled = extract_labeled_dates(text)
    all_dates = [d for d,_ in parse_dates(text)]
    best_date, conf, needs_review, rationale = choose_best_date(et, labeled, all_dates)

    lines = [ln.strip() for ln in (text or "").splitlines() if ln.strip()]
    if et == "HPV_TEST":
        keys = ["hpv", "human papillomavirus", "test", "result", "specimen", "collection", "negative", "positive"]
    elif et == "CO_TEST":
        keys = ["pap", "hpv", "cytology", "co-test", "specimen", "collection", "result"]
    else:  # PAP_SMEAR
        keys = ["pap", "papanicolaou", "cytology", "cervical", "specimen", "collection", "squamous", "interpretation", "result"]
    
    snippet_lines = []
    for ln in lines:
        lnl = ln.lower()
        if any(k in lnl for k in keys):
            snippet_lines.append(ln)
        if len(snippet_lines) >= 8:
            break
    snippet = (" ".join(snippet_lines) or (text or "")[:600]).replace(chr(10), " ")

    # If text was minimal (likely failed OCR), mark for review and note filename-based classification
    text_minimal = len(text.strip()) < 100
    if text_minimal:
        needs_review = True
        conf = min(conf, 0.6)
        if best_date:
            rationale += f"; text extraction minimal ({len(text.strip())} chars), classified by filename"
        else:
            rationale = f"text extraction minimal ({len(text.strip())} chars), classified by filename, no valid date"

    # If no confident collection date, flag for review
    if best_date is None or conf < 0.8:
        needs_review = True

    return ScreeningEvent(
        patient_id=pid,
        event_type=et,
        event_date=best_date,
        source="pdf",
        confidence=float(conf),
        needs_review=bool(needs_review),
        evidence={
            "file": os.path.basename(pdf_path),
            "rationale": rationale,
            "snippet": snippet,
            "dates_by_label": labeled,
            "text_length": len(text),
        }
    )

In [53]:
# Extract events from PDFs (fast text extraction with OCR fallback)
pdf_rows=[]
for pdf_path in PDF_PATHS:
    text = extract_first_page_text(pdf_path)
    ev = extract_event_from_pdf(pdf_path, text)
    pdf_rows.append(ev.__dict__)

df_pdf = pd.DataFrame(pdf_rows)
df_pdf

,patient_id,event_type,event_date,source,confidence,needs_review,evidence
0,03,PAP_SMEAR,2023-08-21,pdf,0.90,False,"{'file': '03_03_1970_pap.pdf', 'rationale': 'p..."
1,117433,CO_TEST,2024-10-21,pdf,0.90,False,"{'file': '117433_pap.pdf', 'rationale': 'picke..."
2,118815,CO_TEST,2022-11-29,pdf,0.90,False,"{'file': '118815_pap.pdf', 'rationale': 'picke..."
3,135613,PAP_SMEAR,2024-06-27,pdf,0.55,True,"{'file': '135613_pap_note.pdf', 'rationale': '..."
4,144061,CO_TEST,2023-07-18,pdf,0.90,False,"{'file': '144061_pap.pdf', 'rationale': 'picke..."
5,151159,CO_TEST,2024-10-01,pdf,0.90,False,"{'file': '151159_pap.pdf', 'rationale': 'picke..."
6,158353,CO_TEST,2024-06-28,pdf,0.90,False,"{'file': '158353_pap.pdf', 'rationale': 'picke..."
7,159266,CO_TEST,2024-04-03,pdf,0.90,False,"{'file': '159266_pap_hpv.pdf', 'rationale': 'p..."
8,176310,CO_TEST,2023-07-24,pdf,0.90,False,"{'file': '176310_pap.pdf', 'rationale': 'picke..."
9,176631,PAP_SMEAR,2025-07-02,pdf,0.55,True,"{'file': '176631_pap_note.pdf', 'rationale': '..."


In [54]:
# Parse cervical screening CSV extract and convert to ScreeningEvent rows
import pandas as pd
import numpy as np
import datetime as dt

def detect_header_row(csv_path: str, max_scan: int = 40) -> int:
    with open(csv_path, 'r', errors='ignore') as f:
        for i,line in enumerate(f):
            if i>max_scan:
                break
            l=line.lower()
            if 'person' in l and ('nbr' in l or '#' in l) and ('mrn' in l or 'enc' in l or 'date' in l):
                return i
    return 0

# Check if CSV exists before processing
if not os.path.exists(CERVICAL_CSV_PATH):
    print(f"⚠️ CSV file not found: {CERVICAL_CSV_PATH}")
    print("Creating empty dataframe for CSV events")
    df_csv_events = pd.DataFrame(columns=["patient_id","event_type","event_date","source","confidence","needs_review","evidence"])
else:
    hdr = detect_header_row(CERVICAL_CSV_PATH)
    df_csv = pd.read_csv(CERVICAL_CSV_PATH, header=hdr)
    df_csv.columns = [c.strip() for c in df_csv.columns]

    # Identify key columns
    def find_col(cols, patterns):
        for p in patterns:
            for c in cols:
                if re.search(p, c, re.I):
                    return c
        return None

    pid_col = find_col(df_csv.columns, [r'person\s*nbr', r'person\s*#', r'patient'])
    encdate_col = find_col(df_csv.columns, [r'enc\s*date', r'collection\s*date', r'specimen\s*date', r'collected', r'date'])
    test_col = find_col(df_csv.columns, [r'test', r'order', r'procedure', r'description', r'name'])

    def parse_any_date(x):
        if pd.isna(x): return None
        s=str(x).strip()
        m=DATE_PAT.search(s)
        if not m: 
            return None
        mm,dd,yy = map(int, m.groups())
        iso=_to_iso(mm,dd,yy)
        return iso

    csv_events=[]
    for idx,row in df_csv.iterrows():
        pid = str(row.get(pid_col, '')).strip()
        if not pid or pid.lower()=='nan':
            continue
        ev_date = parse_any_date(row.get(encdate_col, None))
        if not ev_date:
            continue
        test_name = str(row.get(test_col, '')).lower() if test_col else ''
        
        # Classify based on test name
        if 'hpv' in test_name and ('pap' in test_name or 'cytology' in test_name):
            et = 'CO_TEST'
        elif 'hpv' in test_name:
            et = 'HPV_TEST'
        else:
            et = 'PAP_SMEAR'
        
        csv_events.append({
            "patient_id": pid,
            "event_type": et,
            "event_date": ev_date,
            "source": "structured_csv",
            "confidence": 0.95,
            "needs_review": False,
            "evidence": {"source_file": os.path.basename(CERVICAL_CSV_PATH), "row_index": int(idx), "test_name": test_name}
        })

    df_csv_events = pd.DataFrame(csv_events)

df_csv_events.head() if not df_csv_events.empty else df_csv_events, df_csv_events.shape

⚠️ CSV file not found: csv_data/cervical_screening.csv
Creating empty dataframe for CSV events


(Empty DataFrame
 Columns: [patient_id, event_type, event_date, source, confidence, needs_review, evidence]
 Index: [],
 (0, 7))

In [55]:
# Cervical Cancer Screening 2025 rule engine + merge, de-dup, and scoring
import datetime as dt
from dateutil.relativedelta import relativedelta

year_start = dt.date(REPORTING_YEAR,1,1)
year_end = dt.date(REPORTING_YEAR,12,31)

def counts_for_cervical_screening(event_type: str, event_date_iso: str) -> bool:
    """
    UDS 2025 Cervical Cancer Screening Rules:
    - PAP smear alone: Within 3 years (2023-2025)
    - HPV test alone: Within 5 years (2021-2025)
    - Co-testing (PAP + HPV): Within 5 years (2021-2025)
    """
    if not event_date_iso:
        return False
    d = dt.date.fromisoformat(event_date_iso)
    
    if event_type == "PAP_SMEAR":
        # PAP smear: 3 year lookback
        return (year_start - relativedelta(years=2)) <= d <= year_end
    elif event_type in ["HPV_TEST", "CO_TEST"]:
        # HPV and co-testing: 5 year lookback
        return (year_start - relativedelta(years=4)) <= d <= year_end
    
    return False

# Merge PDF and CSV events
df_all = pd.concat([
    df_pdf[["patient_id","event_type","event_date","source","confidence","needs_review","evidence"]],
    df_csv_events[["patient_id","event_type","event_date","source","confidence","needs_review","evidence"]],
], ignore_index=True)

# De-dup: keep highest confidence for same patient/type/date
df_all["dedup_key"] = df_all["patient_id"].astype(str)+"|"+df_all["event_type"].astype(str)+"|"+df_all["event_date"].astype(str)
df_all = (df_all
          .sort_values(["confidence","source"], ascending=[False, True])
          .drop_duplicates("dedup_key", keep="first")
          .drop(columns=["dedup_key"])
         )

df_all["counts_cervical_2025"] = df_all.apply(lambda r: counts_for_cervical_screening(r["event_type"], r["event_date"]), axis=1)
df_all.sort_values(["patient_id","counts_cervical_2025","event_date"], ascending=[True, False, False]).head(30)

C:\Users\jloya\AppData\Local\Temp\ipykernel_9816\3388831079.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([


,patient_id,event_type,event_date,source,confidence,needs_review,evidence,counts_cervical_2025
0,03,PAP_SMEAR,2023-08-21,pdf,0.90,False,"{'file': '03_03_1970_pap.pdf', 'rationale': 'p...",True
1,117433,CO_TEST,2024-10-21,pdf,0.90,False,"{'file': '117433_pap.pdf', 'rationale': 'picke...",True
2,118815,CO_TEST,2022-11-29,pdf,0.90,False,"{'file': '118815_pap.pdf', 'rationale': 'picke...",True
3,135613,PAP_SMEAR,2024-06-27,pdf,0.55,True,"{'file': '135613_pap_note.pdf', 'rationale': '...",True
4,144061,CO_TEST,2023-07-18,pdf,0.90,False,"{'file': '144061_pap.pdf', 'rationale': 'picke...",True
5,151159,CO_TEST,2024-10-01,pdf,0.90,False,"{'file': '151159_pap.pdf', 'rationale': 'picke...",True
6,158353,CO_TEST,2024-06-28,pdf,0.90,False,"{'file': '158353_pap.pdf', 'rationale': 'picke...",True
7,159266,CO_TEST,2024-04-03,pdf,0.90,False,"{'file': '159266_pap_hpv.pdf', 'rationale': 'p...",True
8,176310,CO_TEST,2023-07-24,pdf,0.90,False,"{'file': '176310_pap.pdf', 'rationale': 'picke...",True
9,176631,PAP_SMEAR,2025-07-02,pdf,0.55,True,"{'file': '176631_pap_note.pdf', 'rationale': '...",True


In [56]:
# Pick best numerator evidence per patient (most recent qualifying; tie-break by confidence)
def pick_best(df):
    q = df[df["counts_cervical_2025"]==True].copy()
    if q.empty:
        # keep best available for review (most recent date regardless) if any
        df2 = df.copy()
        df2 = df2[df2["event_date"].notna()]
        if df2.empty:
            return pd.Series({
                "numerator_met": False,
                "best_event_type": None,
                "best_event_date": None,
                "best_source": None,
                "best_confidence": None,
                "needs_review": True,
                "evidence_summary": "no dated cervical screening evidence found",
            })
        df2 = df2.sort_values(["event_date","confidence"], ascending=[False,False]).head(1)
        row = df2.iloc[0]
        return pd.Series({
            "numerator_met": False,
            "best_event_type": row.event_type,
            "best_event_date": row.event_date,
            "best_source": row.source,
            "best_confidence": row.confidence,
            "needs_review": True,
            "evidence_summary": f"Best non-qualifying evidence: {row.event_type} {row.event_date} ({row.source}); {row.evidence.get('file', row.evidence.get('source_file',''))}",
        })
    q = q.sort_values(["event_date","confidence"], ascending=[False,False]).head(1)
    row = q.iloc[0]
    file_or_row = row.evidence.get('file', row.evidence.get('source_file',''))
    snippet = row.evidence.get('snippet','')
    summary = f"{row.event_type} on {row.event_date} via {row.source} ({file_or_row})"
    if snippet:
        summary += f" | snippet: {snippet[:160]}"
    return pd.Series({
        "numerator_met": True,
        "best_event_type": row.event_type,
        "best_event_date": row.event_date,
        "best_source": row.source,
        "best_confidence": row.confidence,
        "needs_review": bool(row.needs_review),
        "evidence_summary": summary,
    })

df_best = df_all.groupby("patient_id", as_index=False).apply(pick_best).reset_index(drop=True)
df_best = df_best.sort_values(["numerator_met","needs_review","best_event_date"], ascending=[False, True, False])

# Save auditor table
os.makedirs("audit_data", exist_ok=True)
out_csv = "audit_data/cervical_screening_2025_audit_table.csv"
df_best.to_csv(out_csv, index=False, escapechar='\\', doublequote=False)
print(f"✓ Saved audit table to: {out_csv}")
out_csv, df_best.head(50)

✓ Saved audit table to: audit_data/cervical_screening_2025_audit_table.csv


C:\Users\jloya\AppData\Local\Temp\ipykernel_9816\1839577460.py:46: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_best = df_all.groupby("patient_id", as_index=False).apply(pick_best).reset_index(drop=True)


('audit_data/cervical_screening_2025_audit_table.csv',
    patient_id  numerator_met best_event_type best_event_date best_source  \
 12      38837           True         CO_TEST      2025-11-18         pdf   
 11      36387           True         CO_TEST      2025-07-11         pdf   
 1      117433           True         CO_TEST      2024-10-21         pdf   
 5      151159           True         CO_TEST      2024-10-01         pdf   
 6      158353           True         CO_TEST      2024-06-28         pdf   
 10     186419           True         CO_TEST      2024-06-14         pdf   
 7      159266           True         CO_TEST      2024-04-03         pdf   
 0          03           True       PAP_SMEAR      2023-08-21         pdf   
 8      176310           True         CO_TEST      2023-07-24         pdf   
 4      144061           True         CO_TEST      2023-07-18         pdf   
 2      118815           True         CO_TEST      2022-11-29         pdf   
 14      97024       

In [57]:
# Summary statistics
print("="*80)
print("CERVICAL CANCER SCREENING UDS 2025 SUMMARY")
print("="*80)
print(f"\nTotal unique patients: {len(df_best)}")
print(f"Numerator met: {df_best['numerator_met'].sum()} ({df_best['numerator_met'].sum()/len(df_best)*100:.1f}%)")
print(f"Needs review: {df_best['needs_review'].sum()} ({df_best['needs_review'].sum()/len(df_best)*100:.1f}%)")

print("\n" + "-"*80)
print("By Event Type:")
print("-"*80)
event_counts = df_best[df_best['numerator_met']==True]['best_event_type'].value_counts()
for event_type, count in event_counts.items():
    print(f"  {event_type}: {count} patients")

print("\n" + "-"*80)
print("By Source:")
print("-"*80)
source_counts = df_best[df_best['numerator_met']==True]['best_source'].value_counts()
for source, count in source_counts.items():
    print(f"  {source}: {count} patients")

print("\n" + "-"*80)
print("Confidence Distribution (qualifying events):")
print("-"*80)
qualifying = df_best[df_best['numerator_met']==True]
if not qualifying.empty:
    print(f"  Mean confidence: {qualifying['best_confidence'].mean():.2f}")
    print(f"  Min confidence: {qualifying['best_confidence'].min():.2f}")
    print(f"  Max confidence: {qualifying['best_confidence'].max():.2f}")
    print(f"  High confidence (≥0.85): {(qualifying['best_confidence']>=0.85).sum()} patients")
    print(f"  Medium confidence (0.65-0.84): {((qualifying['best_confidence']>=0.65) & (qualifying['best_confidence']<0.85)).sum()} patients")
    print(f"  Low confidence (<0.65): {(qualifying['best_confidence']<0.65).sum()} patients")

CERVICAL CANCER SCREENING UDS 2025 SUMMARY

Total unique patients: 15
Numerator met: 15 (100.0%)
Needs review: 4 (26.7%)

--------------------------------------------------------------------------------
By Event Type:
--------------------------------------------------------------------------------
  CO_TEST: 10 patients
  PAP_SMEAR: 5 patients

--------------------------------------------------------------------------------
By Source:
--------------------------------------------------------------------------------
  pdf: 15 patients

--------------------------------------------------------------------------------
Confidence Distribution (qualifying events):
--------------------------------------------------------------------------------
  Mean confidence: 0.81
  Min confidence: 0.55
  Max confidence: 0.90
  High confidence (≥0.85): 11 patients
  Medium confidence (0.65-0.84): 1 patients
  Low confidence (<0.65): 3 patients


In [58]:
# DEBUG: Check what pdfplumber extracts vs what's actually in a sample PDF
# Update this path to point to an actual PAP smear PDF in your data
test_pdf = "pdf_data/sample_pap.pdf"

if os.path.exists(test_pdf):
    print("="*80)
    print("PDFPLUMBER EXTRACTION:")
    print("="*80)
    try:
        with pdfplumber.open(test_pdf) as pdf:
            print(f"Total pages: {len(pdf.pages)}")
            for i, page in enumerate(pdf.pages[:3]):  # Check first 3 pages
                text = page.extract_text()
                print(f"\n--- Page {i+1} (length: {len(text) if text else 0}) ---")
                print(text[:1000] if text else "No text extracted")
                print()
    except Exception as e:
        print(f"Error: {e}")

    print("\n" + "="*80)
    print(f"TRYING OCR with POPPLER_PATH={POPPLER_PATH}:")
    print("="*80)
    try:
        pages = convert_from_path(test_pdf, dpi=150, first_page=1, last_page=1, poppler_path=POPPLER_PATH)
        if pages:
            ocr_text = pytesseract.image_to_string(pages[0])
            print(f"✓ OCR SUCCESS! Extracted {len(ocr_text)} chars")
            print(f"OCR text (first 1000 chars):\n{ocr_text[:1000]}")
    except Exception as e:
        print(f"✗ OCR failed: {e}")
else:
    print(f"⚠️ Test PDF not found: {test_pdf}")
    print("Update the 'test_pdf' variable to point to an actual PAP smear PDF in your pdf_data folder")

⚠️ Test PDF not found: pdf_data/sample_pap.pdf
Update the 'test_pdf' variable to point to an actual PAP smear PDF in your pdf_data folder
